In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from viz import lineplot
save_kwargs = dict(dpi=300, bbox_inches="tight", transparent=True)

In [2]:
import msgpack
with open("_viz_embeddings.msgpack", "rb") as f:
    rares = msgpack.load(f)
    


In [3]:
# {r["meta"]["num_ans"] for r in rares}

In [4]:
def verify(meta, **kwargs):
    for k, v in kwargs.items():
        if callable(v):
            if not v(meta):
                return False
        elif meta[k] != v:
            return False
    return True

In [5]:
import targets
T = targets.get(90)

In [38]:
from copy import deepcopy
# bad: 90 * 190, CKL, n_search=10_000
# good: 90 * 160, CKL, n_search=10_000

num_ans = 90 * 160
filter = dict(
    noise_model="CKL",
    num_ans=num_ans,
    sampling="salmon-tste",
    _=lambda meta: ("TSTE-n_search=3000-" in meta["meta__fname"])
)
mrare = [r for r in rares if verify(r["meta"], **filter)]
assert len(mrare) == 1, len(mrare)
em = mrare[0]["embedding"]

df1 = pd.DataFrame(em, columns=["x", "y"])
df1["smoothness"] = T
df1["num_ans"] = deepcopy(filter["num_ans"])

filter["num_ans"] = 90 * 190
mrare = [r for r in rares if verify(r["meta"], **filter)]
assert len(mrare) == 1, len(mrare)
em = mrare[0]["embedding"]

df2 = pd.DataFrame(em, columns=["x", "y"])
df2["smoothness"] = T
df2["num_ans"] = deepcopy(filter["num_ans"])

In [39]:
df2.head()

,x,y,smoothness,num_ans
0,0.405520,5.873869,22,17100
1,0.062623,5.294353,36,17100
2,0.186110,5.206673,39,17100
3,0.467721,5.199568,40,17100
4,0.043861,4.723218,50,17100


In [40]:
df = pd.concat((df1, df2))
print(df.num_ans.unique())
df.head()

[14400 17100]


,x,y,smoothness,num_ans
0,-3.262579,5.342608,22,14400
1,-3.145626,4.897232,36,14400
2,-3.000186,4.893074,39,14400
3,-2.904901,5.007272,40,14400
4,-2.894516,4.584447,50,14400


In [41]:
import altair as alt
alt.Chart(df).mark_circle(size=200).encode(
    x=alt.X("x", scale=alt.Scale(), axis=alt.Axis(labels=False)),
    y=alt.Y("y", scale=alt.Scale(), axis=alt.Axis(labels=False)),
    color="smoothness",
    row="num_ans",
)

alt.Chart(...)